# Task 1

#### 1) build colleration matrix and filter results with corellation > 95 or < -.95

In [2]:
import pandas as pd

df = pd.read_csv('sp500_27270.csv')

corr = df.corr()
filtered_companies = [company for company in corr['SP500'].keys() if corr['SP500'][company] > 0.95 or corr['SP500'][company] < -0.95]
# print(filtered_companies)

df_filtered = df.filter(filtered_companies)

# print(df_filtered)
print(len(filtered_companies))
print(df_filtered.shape)

72
(1260, 72)


#### 2) split dataset to training and testing

In [15]:
TRAINING_PERCENTAGE = 0.8
TESTING_PERCENTAGE = 0.2
# VALIDATION_PERCENTAGE = 0.5 

from sklearn.model_selection import train_test_split

train, test = train_test_split(df_filtered, test_size=TESTING_PERCENTAGE)

print("Training chunk length: ", len(train))
print("Testing chunk length: ", len(test))

Training chunk length:  1008
Testing chunk length:  252


#### 3) Choose scoring metric 

In [73]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(early_stopping=True, max_iter=10000, random_state=1, learning_rate='adaptive')

# print(type(test['SP500']))
# print(list(test['SP500']))

Y_train = list(train['SP500'])
X_train = train.drop(columns=['SP500'])



model.fit(X_train, Y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=10000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

#### 4) Test fitted model

In [74]:
# from sklearn import metrics
from sklearn.metrics import accuracy_score

Y_test = list(test['SP500'])
X_test = test.drop(columns=['SP500'])


# predictions = model.predict(X_test)

# acc = accuracy_score(list(Y_test), list(predictions))

score = model.score(X_test, Y_test)

print("Accuracy: ", score)

Accuracy:  0.9933791856317532


#### Use GridSearch

In [71]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'solver': ['adam', 'sgd'],
#     'activation': ['relu', 'tanh'],
    'hidden_layer_sizes': [(15, 2), (10, 2), (20, 2), (30, 2)]
}

clf = GridSearchCV(model, param_grid=param_grid, cv=3)
clf.fit(X_train,Y_train)
print(clf.best_params_)

{'hidden_layer_sizes': (10, 2), 'solver': 'adam'}


In [72]:
score = clf.score(X_test, Y_test)

print("Accuracy: ", score)

Accuracy:  0.9864448249655801


# Task 2

#### 1) get most correlated company

In [36]:
max_corr = -100
max_company = ''

for i in corr['SP500'].keys():
    if corr['SP500'][i] > max_corr and i != 'SP500':
        max_corr = corr['SP500'][i]
        max_company = i
        
        
print(max_company)
print(max_corr)

max_company_data = df[max_company]
# print(max_company_data)

APH
0.9883608178608217


#### 2) prepare input and output data set of days using shifting

In [37]:
# print(type(max_company_data))
# print(max_company_data)
# print(max_company_data.shift(-1))
# print(len(max_company_data))

X_data = []
Y_data = []

for iter in range(len(max_company_data) - 9):
    # get first 5 days for input
    X_data.append(list(max_company_data)[:5])
    # get following for days for output
    Y_data.append(list(max_company_data)[5:9])
    # shift data upwards by one step
    max_company_data = max_company_data.shift(-1)
    
print(X_data)
    


[[36.2702, 35.8013, 36.2127, 36.4472, 36.8634], [35.8013, 36.2127, 36.4472, 36.8634, 36.8873], [36.2127, 36.4472, 36.8634, 36.8873, 37.0548], [36.4472, 36.8634, 36.8873, 37.0548, 37.8203], [36.8634, 36.8873, 37.0548, 37.8203, 38.8106], [36.8873, 37.0548, 37.8203, 38.8106, 39.3896], [37.0548, 37.8203, 38.8106, 39.3896, 39.3943], [37.8203, 38.8106, 39.3896, 39.3943, 39.2269], [38.8106, 39.3896, 39.3943, 39.2269, 39.0212], [39.3896, 39.3943, 39.2269, 39.0212, 38.891999999999996], [39.3943, 39.2269, 39.0212, 38.891999999999996, 38.8824], [39.2269, 39.0212, 38.891999999999996, 38.8824, 38.6241], [39.0212, 38.891999999999996, 38.8824, 38.6241, 38.7245], [38.891999999999996, 38.8824, 38.6241, 38.7245, 38.4757], [38.8824, 38.6241, 38.7245, 38.4757, 38.4135], [38.6241, 38.7245, 38.4757, 38.4135, 38.2413], [38.7245, 38.4757, 38.4135, 38.2413, 38.4135], [38.4757, 38.4135, 38.2413, 38.4135, 38.1791], [38.4135, 38.2413, 38.4135, 38.1791, 38.6958], [38.2413, 38.4135, 38.1791, 38.6958, 38.404], [38.4

#### 3) split dataset into test and train

In [39]:
TRAINING_PERCENTAGE = 0.8
TESTING_PERCENTAGE = 0.2
# VALIDATION_PERCENTAGE = 0.5 

from sklearn.model_selection import train_test_split

train, test = train_test_split(df_filtered, test_size=TESTING_PERCENTAGE)
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=TESTING_PERCENTAGE)

print("Training chunk length: ", len(X_train))
print("Testing chunk length: ", len(X_test))
# print(X_train)

Training chunk length:  1000
Testing chunk length:  251


#### 4) Create model and it 

In [41]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(early_stopping=True, max_iter=10000, random_state=1, learning_rate='adaptive')

model.fit(X_train, y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='adaptive',
       learning_rate_init=0.001, max_iter=10000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

#### 5) Evaluate model

In [42]:
score = model.score(X_test, y_test)

print("Accuracy: ", score)

Accuracy:  0.994349635826715


# Task 3

## Some context on the TfidVectorizer and text handing in sklearn

* To represent a review to a neural network, each word has to be expressed as a number. 

* Each word in the review corpus is assigned a unique number. Assume the total number of unique words is feature_count. 

* Each review can then be represented as a vector of length feature_count. 

* Entries in this vector can be either 0 or 1, if the word is present in this review or absent respectively. 

* A more advanced approach will replace the binary numbers above with ratios that emphaise how rare ( and hence meanigful and representative ) the word is across all reviews. For example, a word like "product" will have little use for the neural network in determing whetehr this is a rating-1 or rating-5 review, as this particular word occurs in most reviews, both positive and negative ones. 

* Here we limit the number of feature_count to 40,000 to cut down the training time. 

more info available from 
http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

#### Read dataset and clean

In [2]:
import pandas as pd

reviews = pd.read_json('reviews_digital.json', lines=True)


64706


In [16]:
print(reviews.groupby('overall').count()['asin'])
print()
frames = []

for i in range(1,6):
    frames.append(reviews[(reviews['overall'] == i)][:2500])

balanced_data = pd.concat(frames)

print(len(balanced_data))

overall
1     2791
2     3010
3     6789
4    16536
5    35580
Name: asin, dtype: int64

12500


#### Split data

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

TESTING_PERCENTAGE = 0.2

X_data = np.array(balanced_data['reviewText'])

stop_words = set(stopwords.words('english')) 
X_data_new = []
for text in X_data:
    word_tokens = word_tokenize(text)
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    X_data_new.append(' '.join(filtered_sentence))
  

Y_data = np.array(balanced_data['overall'])


X_train, X_test, y_train, y_test = train_test_split(X_data_new, Y_data, test_size=TESTING_PERCENTAGE)


#### Calculate tfidf scores

In [35]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

#You need to define X_train and X_test
tfidf_vectorizer = TfidfVectorizer(max_features=40000)
X_train_transformed = tfidf_vectorizer.fit_transform(X_train)
X_test_transformed = tfidf_vectorizer.transform(X_test)

# print(X_train_transformed)

In [65]:
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(early_stopping=True, hidden_layer_sizes=(1024,))

model.fit(X_train_transformed, y_train)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(1024,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [66]:
print(model.score(X_test_transformed, y_test))

0.6264


In [ ]:
from sklearn import model_selection
# seed = 7
# kfold = model_selection.KFold(n_splits=10, random_state=seed)
scoring = 'accuracy'
results = model_selection.cross_val_score(model, X_train_transformed, y_train, cv=3, scoring=scoring, verbose=20)

[CV]  ................................................................


In [70]:
results.mean()
results.std()

0.5972996342489773

In [57]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'solver': ['adam'],
    'activation': ['relu', 'tanh'],
    'hidden_layer_sizes': [(256, 512), (512, 1024)],
    'early_stopping': [True]
}

grid_search = GridSearchCV(model, param_grid=param_grid, verbose=20)

grid_search.fit(X_train_transformed, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] activation=relu, early_stopping=True, hidden_layer_sizes=(256, 512), solver=adam 
[CV]  activation=relu, early_stopping=True, hidden_layer_sizes=(256, 512), solver=adam, score=0.5916041979010495, total=  42.9s
[CV] activation=relu, early_stopping=True, hidden_layer_sizes=(256, 512), solver=adam 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   43.0s remaining:    0.0s


[CV]  activation=relu, early_stopping=True, hidden_layer_sizes=(256, 512), solver=adam, score=0.588058805880588, total=  44.4s
[CV] activation=relu, early_stopping=True, hidden_layer_sizes=(256, 512), solver=adam 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.5min remaining:    0.0s


[CV]  activation=relu, early_stopping=True, hidden_layer_sizes=(256, 512), solver=adam, score=0.5972388955582233, total=  45.2s
[CV] activation=relu, early_stopping=True, hidden_layer_sizes=(512, 1024), solver=adam 


...

[CV]  activation=tanh, early_stopping=True, hidden_layer_sizes=(512, 1024), solver=adam, score=0.5916591659165916, total= 1.1min
[CV] activation=tanh, early_stopping=True, hidden_layer_sizes=(512, 1024), solver=adam 


[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed: 10.8min remaining:    0.0s


[CV]  activation=tanh, early_stopping=True, hidden_layer_sizes=(512, 1024), solver=adam, score=0.5834333733493398, total= 1.2min


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 11.9min remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 11.9min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'solver': ['adam'], 'activation': ['relu', 'tanh'], 'hidden_layer_sizes': [(256, 512), (512, 1024)], 'early_stopping': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=20)

In [59]:
print(grid_search.best_params_)

{'activation': 'tanh', 'early_stopping': True, 'hidden_layer_sizes': (256, 512), 'solver': 'adam'}
